In [2]:
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# What exactly is being learned in each chunk? We only take a subset of the edges? 
#is there a way to ensure that same src edges are being used such that the training learns them together for a given target node?

In [5]:
class BatchedMemoryEfficientGATLayer(nn.Module):
    def __init__(self, in_features, edge_features, out_features, num_heads, dropout=0.6, leaky_relu_slope=0.2):
        super(BatchedMemoryEfficientGATLayer, self).__init__()
        self.in_features = in_features
        self.edge_features = edge_features
        self.out_features = out_features
        self.num_heads = num_heads
        self.dropout = dropout
        
        # Initial linear layer on node attributes
        # self.initial_linear = nn.Linear(in_features, in_features)
        
        # Increased projection size (2x out_features)
        self.W = nn.Linear(in_features, num_heads * 1 * out_features, bias=False)
        
        # Adjust attention parameter size for increased projection
        self.a = nn.Parameter(torch.empty(size=(num_heads, 2 * (1 * out_features) + edge_features)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        # Final projection to out_features
        self.final_projection = nn.Linear(1 * out_features, out_features)
        
        self.leaky_relu = nn.LeakyReLU(leaky_relu_slope)

    def _neighborhood_aware_softmax(self, attention_scores, edge_index_chunk, num_nodes):
        batch_size, chunk_size, num_heads = attention_scores.size()
        
        # Initialize the output tensor
        normalized_scores = torch.zeros_like(attention_scores)
        
        # Process each batch separately
        for b in range(batch_size):
            # Create tensors for source and destination nodes
            src_nodes = edge_index_chunk[b, 0]
            dst_nodes = edge_index_chunk[b, 1]
            
            # Compute max for numerical stability
            max_scores = torch.zeros(num_nodes, num_heads, device=attention_scores.device)
            max_scores.index_reduce_(0, dst_nodes, attention_scores[b], 'amax')
            
            # Compute exponentials
            exp_scores = torch.exp(attention_scores[b] - max_scores[dst_nodes])
            
            # Compute sum of exponentials for each destination node
            sum_exp = torch.zeros(num_nodes, num_heads, device=attention_scores.device)
            sum_exp.index_add_(0, dst_nodes, exp_scores)
            
            # Compute normalized scores
            normalized_scores[b] = exp_scores / sum_exp[dst_nodes].clamp(min=1e-12)
        
        return normalized_scores

    # def _neighborhood_aware_softmax(self, attention_scores, edge_index_chunk, num_nodes):
    #     batch_size, chunk_size, num_heads = attention_scores.size()
        
    #     # Compute max for numerical stability
    #     max_scores = torch.zeros(batch_size, num_nodes, num_heads, device=attention_scores.device)
    #     max_scores.scatter_reduce_(1, edge_index_chunk[:, 1].unsqueeze(-1).expand(-1, -1, num_heads), 
    #                                attention_scores, reduce='amax')
        
    #     # Compute exponentials
    #     exp_scores = torch.exp(attention_scores - max_scores.gather(1, edge_index_chunk[:, 1].unsqueeze(-1).expand(-1, -1, num_heads)))
        
    #     # Compute sum of exponentials for each destination node
    #     sum_exp = torch.zeros(batch_size, num_nodes, num_heads, device=attention_scores.device)
    #     sum_exp.scatter_add_(1, edge_index_chunk[:, 1].unsqueeze(-1).expand(-1, -1, num_heads), exp_scores)
        
    #     # Compute normalized scores
    #     normalized_scores = exp_scores / (sum_exp.gather(1, edge_index_chunk[:, 1].unsqueeze(-1).expand(-1, -1, num_heads)) + 1e-12)
        
    #     return normalized_scores

    def forward(self, x, edge_index, edge_attr, edge_distance):
        batch_size, N, _ = x.size()
        
        # Apply initial linear layer
        # x = self.initial_linear(x)
        
        # Increased projection
        h = self.W(x).view(batch_size, N, self.num_heads, -1)  # [batch_size, N, num_heads, 2*out_features]
        
        # Combine edge_attr and edge_distance
        edge_features = torch.cat([edge_attr.unsqueeze(-1), edge_distance.unsqueeze(-1)], dim=-1)
        
        # Process edges in chunks to save memory
        chunk_size = 8 * 1_000  # Adjust this based on available memory
        num_edges = edge_index.size(2)
        h_prime = torch.zeros_like(h)
        
        for i in range(0, num_edges, chunk_size):
            edge_index_chunk = edge_index[:, :, i:i+chunk_size]
            edge_features_chunk = edge_features[:, i:i+chunk_size, :].unsqueeze(1).permute([0, 2, 1, 3])
            
            h_src = h.gather(1, edge_index_chunk[:, 0, :].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, self.num_heads, h.size(-1)))
            h_dst = h.gather(1, edge_index_chunk[:, 1, :].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, self.num_heads, h.size(-1)))
            
            # Compute attention coefficients
            # Compute attention coefficients
            edge_h = torch.cat([h_src, h_dst, edge_features_chunk], dim=-1)
            edge_e = self.leaky_relu((self.a.view(1, 1, self.num_heads, -1) * edge_h).sum(dim=-1))
            attention = self._neighborhood_aware_softmax(edge_e, edge_index_chunk, N)
            
            # Apply dropout to attention weights
            attention = F.dropout(attention, self.dropout, training=self.training)
            
            # Apply attention coefficients
            weighted_features = attention.unsqueeze(-1) * h_src
            for b in range(batch_size):
                h_prime[b].scatter_add_(0, 
                    edge_index_chunk[b, 1].view(-1, 1, 1).expand(-1, self.num_heads, h.size(-1)), 
                    weighted_features[b]
                )
        
        # Average over heads and apply final projection
        h_prime = h_prime.mean(dim=2)  # [batch_size, N, 2*out_features]
        return self.final_projection(h_prime)  # [batch_size, N, out_features]

class BatchedMemoryEfficientGAT(nn.Module):
    def __init__(self, num_features, edge_features, hidden_size, num_classes, num_heads, num_layers=2):
        super(BatchedMemoryEfficientGAT, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        
        self.gat_layers.append(BatchedMemoryEfficientGATLayer(num_features, edge_features, hidden_size, num_heads))
        for _ in range(num_layers - 2):
            self.gat_layers.append(BatchedMemoryEfficientGATLayer(hidden_size, edge_features, hidden_size, num_heads))
        self.gat_layers.append(BatchedMemoryEfficientGATLayer(hidden_size, edge_features, num_classes, 1))
        
        self.dropout = nn.Dropout(0.6)

    def forward(self, x, edge_index, edge_attr, edge_distance):
        for i in range(self.num_layers - 1):
            x = F.elu(self.gat_layers[i](x, edge_index, edge_attr, edge_distance))
            x = self.dropout(x)
        
        x = self.gat_layers[-1](x, edge_index, edge_attr, edge_distance)
        return F.log_softmax(x, dim=-1)

In [6]:
from fluid_simulation.utils import create_grid_graph_with_angles

In [7]:
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd

class GridDatasetGNN(Dataset):
    def __init__(self, df, feature_cols, target_cols, height, width):
        """
        Args:
            df (pd.DataFrame): DataFrame containing simulation data with columns ['simulation_id', 'timestep', 'row', 'col', ...].
            feature_cols (list): List of column names to be used as input features.
            target_cols (list): List of column names to be used as target features.
            height (int): Number of rows in the grid.
            width (int): Number of columns in the grid.
        """
        super(GridDatasetGNN, self).__init__()
        self.df = df
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        self.height = height
        self.width = width
        self.graph = create_grid_graph_with_angles(height, width)
        self.data_list = self.process_data()

    def process_data(self):
        data_list = []
        grouped = self.df.groupby(['simulation_id', 'timestep'])
        for (sim_id, timestep), group in grouped:
            assert group[['row', 'col']].duplicated().sum() == 0, "Duplicate (row, col) found."
            
            group = group.sort_values(['row', 'col']).reset_index(drop=True)
            
            features = group[self.feature_cols].values.astype(np.float32)
            x = torch.tensor(features, dtype=torch.float)
            y = torch.tensor(group[self.target_cols].values.astype(np.float32), dtype=torch.float)
            
            data = {
                'x': x,
                'edge_index': self.graph['edge_index'],
                'edge_attr': self.graph['edge_attr'],
                'edge_distance': self.graph['edge_distance'],
                'y': y
            }
            data_list.append(data)
        return data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

In [8]:
import pandas as pd
from fluid_simulation.models_v2 import CNN, GridGNNWithAngles
from fluid_simulation.utils import prepare_data, calculate_deltas, prepare_data_v2

In [9]:
csv_file = '../../combined_mac_data_with_deltas.csv'

timestep_n_rows = 14400
n_steps = 100
df = pd.read_csv(csv_file, nrows=timestep_n_rows * n_steps)
df2 = pd.read_csv(csv_file, nrows=timestep_n_rows * n_steps, skiprows=timestep_n_rows * 100)

df2.columns = df.columns
print(df.shape)
print(df.shape[0] // 51_200)
df = pd.concat([df, df2])

(1440000, 19)
28


In [10]:
float_columns = df.select_dtypes(include=['float64', 'float32']).columns
df[float_columns] = (df[float_columns] - df[float_columns].mean()) / df[float_columns].std()

In [12]:
# df[(df.row > 60) & (df.row < 80) & (df.density > 0.2)].head(30)

In [13]:
device = torch.device("mps")

In [14]:
df, input_cols, target_cols = prepare_data_v2(df, target_pattern="_next", input_pattern_filter_2="delta")

Input columns: ['u', 'v', 'density', 'is_fluid', 'border']
Target columns: ['u_next', 'v_next', 'density_next', 'is_fluid_next']


In [16]:
df[(df.row > 60) & (df.row < 80) & (df.density > 0.2)].head(10)

,iter,time,timestep,row,col,u,v,density,pressure,is_fluid,simulation_id,iter_next,time_next,timestep_next,u_next,v_next,density_next,pressure_next,is_fluid_next,border
511324,750,-0.502320,35,61,4,-1.187416,4.177571,0.201910,0.865086,1,0,NaN,-0.489898,-0.489898,-1.138904,4.185802,0.260399,0.771701,NaN,0.0
511325,750,-0.502320,35,61,5,-0.888408,4.820344,0.210081,1.415486,1,0,NaN,-0.489898,-0.489898,-0.830273,4.833810,0.283502,1.312208,NaN,0.0
525724,750,-0.467677,36,61,4,-1.141350,4.206091,0.262861,0.777781,1,0,NaN,-0.454905,-0.454905,-1.087848,4.201270,0.324587,0.680808,NaN,0.0
525725,750,-0.467677,36,61,5,-0.831515,4.857214,0.286056,1.319596,1,0,NaN,-0.454905,-0.454905,-0.767477,4.852431,0.364575,1.210543,NaN,0.0
540123,750,-0.433034,37,61,3,-1.453723,3.557919,0.217778,0.250830,1,0,NaN,-0.419912,-0.419912,-1.404165,3.555749,0.267096,0.171317,NaN,0.0
540124,750,-0.433034,37,61,4,-1.090095,4.221633,0.327305,0.686668,1,0,NaN,-0.419912,-0.419912,-1.032777,4.204574,0.391496,0.586519,NaN,0.0
540125,750,-0.433034,37,61,5,-0.768474,4.875925,0.367452,1.217685,1,0,NaN,-0.419912,-0.419912,-0.700162,4.854117,0.449836,1.103321,NaN,0.0
540126,750,-0.433034,37,61,6,-0.553296,5.460904,0.280671,1.810637,1,0,NaN,-0.419912,-0.419912,-0.473742,5.458790,0.378257,1.693235,NaN,0.0
554523,750,-0.398392,38,61,3,-1.407646,3.573008,0.269585,0.175943,1,0,NaN,-0.384920,-0.384920,-1.354625,3.561823,0.320596,0.094979,NaN,0.0
554524,750,-0.398392,38,61,4,-1.034809,4.224953,0.394480,0.592151,1,0,NaN,-0.384920,-0.384920,-0.974810,4.196581,0.460309,0.489364,NaN,0.0


In [17]:
dataset_gnn = GridDatasetGNN(
    df=df, feature_cols=input_cols, target_cols=target_cols, height=160,width=160)

In [18]:
import os

# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"

In [19]:
batch_size = 6

model_gnn = BatchedMemoryEfficientGAT(
    num_features=len(input_cols),
    num_classes=len(target_cols),
    edge_features=2,
    hidden_size=16,
    num_heads=1,
).to(device)

In [20]:
loader_gnn = DataLoader(dataset_gnn, batch_size=batch_size, shuffle=True)
# Example: Iterate through the DataLoader
for batch in loader_gnn:
    # batch.x: [batch_size * num_nodes, in_features]
    # batch.edge_index: [2, batch_size * 4 * num_nodes]
    # batch.y: [batch_size * num_nodes, target_features]
    print(batch.keys())
    x = batch['x'].to(device)
    edge_index = batch['edge_index'].to(device)
    edge_attr = batch['edge_attr'].to(device)
    edge_distance = batch['edge_distance'].to(device)
    output = model_gnn(x, edge_index, edge_attr, edge_distance)
    print(edge_attr.shape)
    # Compute loss, backpropagate, etc.
    print(output.shape)
    break  # Remove this to iterate through the entire dataset
print('---- GNN is Valid ----')

dict_keys(['x', 'edge_index', 'edge_attr', 'edge_distance', 'y'])


/var/folders/0k/6kqmsx2j5cqbycjcbrjng36h0000gn/T/ipykernel_37329/1371692449.py:39: UserWarning: The operator 'aten::index_reduce.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  max_scores.index_reduce_(0, dst_nodes, attention_scores[b], 'amax')


IndexError: index out of range in self

In [19]:
import argparse
import pandas as pd
import torch
import torch.nn
import os

from torch.optim.lr_scheduler import StepLR
from torch_geometric.loader import DataLoader

def train_model(model, dataloader, num_epochs=10, learning_rate=0.001, device=None, model_type="gnn"):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Add LR scheduler
    scheduler = StepLR(optimizer, step_size=4, gamma=0.92)  # Reduce LR by factor of 0.1 every 5 epochs
    
    model = model.to(device)
    print(f'Starting training: {next(model.parameters()).device}')
    print('params')
    print(sum(p.numel() for p in model_gnn.parameters() if p.requires_grad))
    print(sum(p.numel() for p in model.gat_layers[0].parameters() if p.requires_grad))
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        i = 0
        for batch in dataloader:
            x = batch['x'].to(device)
            edge_index = batch['edge_index'].to(device)
            edge_attr = batch['edge_attr'].to(device)
            edge_distance = batch['edge_distance'].to(device)
            output = model_gnn(x, edge_index, edge_attr, edge_distance)
            # Assume targets are binary (0 or 1)
            loss = criterion(batch['y'].to(device), output)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Step the scheduler
        scheduler.step()
        
        epoch_loss = running_loss / len(dataloader)
        # if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")
    
    print('Finished training')
    return model

In [20]:
n_epochs = 25

In [21]:
gnn_trained = train_model(
    model_gnn, 
    loader_gnn, 
    num_epochs=n_epochs, 
    learning_rate=0.001, 
    device=device, 
    model_type="gnn"
)

Starting training: mps:0
params
480
386
Epoch 1/25, Loss: 491.7283, LR: 0.001000
Epoch 2/25, Loss: 453.2777, LR: 0.001000
Epoch 3/25, Loss: 458.7526, LR: 0.001000
Epoch 4/25, Loss: 460.1987, LR: 0.000920
Epoch 5/25, Loss: 466.4615, LR: 0.000920


KeyboardInterrupt: 